## Create POSCAR

Zincblende ZnO, lattice constant $a=4.579$ Angstrom

In [1]:
from mykit.vasp.poscar import poscar

zZnO = poscar.zincblende("Zn", "O", a=4.579, primitive=True)

In [2]:
# print zZnO to directly see the POSCAR
print(zZnO)

Zincblende ZnO
1.000000
    0.00000000    2.28950000    2.28950000
    2.28950000    0.00000000    2.28950000
    2.28950000    2.28950000    0.00000000
Zn O
1 1
Direct
    0.000000000     0.000000000     0.000000000 #Zn
    0.250000000     0.250000000     0.250000000 #O


In [3]:
# export to POSCAR
zZnO.write()

## Export POTCAR from POSCAR

The atom types in POSCAR can be extracted by `atomTypes` property (in order)

In [4]:
zZnO.atomTypes

['Zn', 'O']

Parse it (expanded by `*`) to `potcar_search` to create an instance to search POTCARs for "Zn" and "O"

In [5]:
from mykit.vasp.potcar import potcar_search

pts = potcar_search(*zZnO.atomTypes, usegw=False) # usegw set True to search PAW with GW suffix only
# print the names of POTCARs to search
pts.names

('Zn', 'O')

*N.B. You need first setup* `VASP_POT_PBE` *environment variable to your local PAW PBE directory to make the following work for you*

Search the PAW PBE direcotry for available "Zn" and "O" POTCARs, together with their `ENMIN`s and `ENMAX`s

In [6]:
pts.search(xc="PBE")

('/Users/stevezhang/software/sci/vasp/vasppot-5.4/potpaw_PBE/',
 {'Zn': {'Zn_GW': (246.143, 328.191),
   'Zn': (207.542, 276.723),
   'Zn_sv_GW': (301.249, 401.665)},
  'O': {'O_GW': (310.976, 414.635),
   'O_s': (212.14, 282.853),
   'O_h': (500.0, 700.0),
   'O_s_GW': (250.998, 334.664),
   'O_GW_new': (325.824, 434.431),
   'O': (300.0, 400.0),
   'O_h_GW': (574.139, 765.519)}})

After searching, you may specify the names of POTCARs, and export it to current directory.

In [7]:
# Use Zn_sv_GW and O_GW
pts.names = ('Zn_sv_GW', 'O_GW')
pts.export(xc="PBE") # return the searching path of PAW PBE

'/Users/stevezhang/software/sci/vasp/vasppot-5.4/potpaw_PBE/'

## Create KPOINTS

In [8]:
from mykit.vasp.kpoints import kpoints

### Automatic kgrids with division

A k-point grid with divisions along each reciprocal vector is often used for SCF and DOS calculation. It can be created by the following two line.

In [9]:
kpt = kpoints(kgrid=[4,4,4], kmode="G")
# preview the KPOINTS
print(kpt)

KPOINTS generated by mykit
0
Gamma
 4  4  4
 0  0  0


A Gamma-centered KPOINTS is setup, and you may export it to the currenty directory

In [10]:
# export to KPOINTS
kpt.write()

### K-point path for band structure

For band structure calculation, a kpath is required, which has explicit coordinates of turning points in terms of reciprocal lattice vectors. For example, KPOINTS for computing band structure along $\Gamma$-L-X in the Brillouin zone of cF lattice looks like

```
kpath
15
line
reciprocal
0.00 0.00 0.00 1 # GM
0.50 0.50 0.50 1 # L

0.50 0.50 0.50 1 # L
0.50 0.00 0.50 1 # X
```

`kpoints` class can accept a dictionary for `kpath` keyword argument to create such KPOINTS file. It should have two key, namely `sysmbols` and `coordinates`, such as

In [11]:
kpathDict = {
    "symbols": ["GM", "L", "L", "X"],
    "coordinates": 
        [[0.0,0.0,0.0],
         [0.5,0.5,0.5],
         [0.5,0.5,0.5],
         [0.5,0.0,0.5]],
}

Then we create `kpoints` instance with `kmode`, `kpath` and `kdense` keyword

In [12]:
kpath = kpoints(kmode="L", kdense=15, kpath=kpathDict)

Preview the output by print it

In [13]:
print(kpath)

KPOINTS generated by mykit
15
Line
Reciprocal
 0.000000  0.000000  0.000000 1 #GM
 0.500000  0.500000  0.500000 1 #L

 0.500000  0.500000  0.500000 1 #L
 0.500000  0.000000  0.500000 1 #X



As you may find, `kpathDict` is just what you have to type manually in the KPOINTS file, making it unnecessary to build an instance to do this. Moreover, you have to know all the symbols and coordinates before hand.

To make it easier to create kpath, a `special_kpoints` class is implemented. It can be instantialized by a `poscar` instance, e.g. `zZnO` we created above

In [14]:
from mykit.core.symmetry import special_kpoints

spk_zb =  special_kpoints.from_cell(zZnO)

Special k-points symbols and their coordinates in reciprocal vectors of primitive cell that are defined for Zincblende (space group 216, oF) can be shown by `spkCoord` attribute

In [15]:
spk_zb.spkCoord

{'GM': [0.0, 0.0, 0.0],
 'X': [0.5, 0.0, 0.5],
 'L': [0.5, 0.5, 0.5],
 'W': [0.5, 0.25, 0.75],
 'K': [0.375, 0.375, 0.75]}

To create a `kpathDict`-like object, you only need to use the `convert_kpath` method with an intuitive kpath string `'GM-L-X'`

In [16]:
kpathDict2 = spk_zb.convert_kpath('GM-L-X')
kpathDict2

{'symbols': ['GM', 'L', 'L', 'X'], 'coordinates': array([[0. , 0. , 0. ],
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5],
        [0.5, 0. , 0.5]])}

It should be noted that the coordinates of special kpoints depend on whether you are using primitive or conventional cell. `convert_kpath` already takes care this for you and converts to conventional coordinates if the input cell is conventional.

Now you can create the KPOINTS file with `kpathDict2` and preview it

In [17]:
kpath = kpoints(kmode="L", kdense=15, kpath=kpathDict2)
print(kpath)

KPOINTS generated by mykit
15
Line
Reciprocal
 0.000000  0.000000  0.000000 1 #GM
 0.500000  0.500000  0.500000 1 #L

 0.500000  0.500000  0.500000 1 #L
 0.500000  0.000000  0.500000 1 #X



Another class method, `get_kpath_from_cell`, skips the instantialization of `special_kpoints` and directly returns the kpath dictionary

In [18]:
kpathDict3 = special_kpoints.get_kpath_from_cell('GM-L-X', zZnO)
kpath = kpoints(kmode="L", kdense=15, kpath=kpathDict3)
print(kpath)

KPOINTS generated by mykit
15
Line
Reciprocal
 0.000000  0.000000  0.000000 1 #GM
 0.500000  0.500000  0.500000 1 #L

 0.500000  0.500000  0.500000 1 #L
 0.500000  0.000000  0.500000 1 #X

